In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, AUC

In [2]:
from scipy.io import loadmat
from scipy.io import savemat
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist
from scipy.spatial.distance import cdist

In [3]:
mat = loadmat('./results_dblp_2.mat')
features_matrix = mat['embs']
#print(type(features_matrix))
#print(features_matrix[:10,:10])

mat = loadmat("../data/dblp.mat")
A = mat["network"]
#graph = sparse2graph(A)
labels_matrix = mat["group"]
labels_count = labels_matrix.shape[1]

#print(labels_matrix)

#print(type(labels_matrix))

groups = labels_matrix.toarray()
A = A.toarray()
X = features_matrix
print(type(groups[:5, :5]))
print(groups[:5, :5])
print(A[:5, :5])

<class 'numpy.ndarray'>
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 1.]
 [1. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [4]:
# Number of nodes
num_nodes = 13326

# Number of features per node
num_features = 128

# Generate random features
#X = np.random.rand(num_nodes, num_features)

print(X[1, :])
print(groups[1, :])

# Generate a random adjacency matrix
#A = np.random.randint(2, size=(num_nodes, num_nodes))

print(A)


[4.779e+03 2.000e+00 4.160e+02 4.779e+03 2.000e+00 4.160e+02 4.779e+03
 4.779e+03 4.779e+03 4.779e+03 3.498e+03 4.779e+03 3.497e+03 2.000e+00
 4.779e+03 4.779e+03 4.779e+03 4.160e+02 3.497e+03 4.779e+03 4.779e+03
 2.000e+00 3.498e+03 3.498e+03 3.497e+03 4.160e+02 4.779e+03 4.779e+03
 2.000e+00 2.000e+00 4.779e+03 4.779e+03 2.000e+00 2.000e+00 4.779e+03
 4.160e+02 3.497e+03 2.000e+00 2.000e+00 4.160e+02 2.000e+00 4.779e+03
 2.000e+00 3.498e+03 4.779e+03 4.779e+03 4.160e+02 4.779e+03 4.160e+02
 2.000e+00 2.000e+00 3.497e+03 4.779e+03 2.000e+00 4.779e+03 4.779e+03
 4.779e+03 4.779e+03 4.779e+03 4.779e+03 3.498e+03 4.779e+03 4.779e+03
 4.779e+03 4.160e+02 3.497e+03 4.779e+03 2.000e+00 4.779e+03 4.779e+03
 4.779e+03 2.000e+00 4.779e+03 4.779e+03 2.000e+00 3.498e+03 4.779e+03
 4.779e+03 4.779e+03 2.000e+00 2.000e+00 4.160e+02 2.000e+00 4.779e+03
 4.779e+03 3.497e+03 3.498e+03 4.779e+03 4.160e+02 2.000e+00 3.498e+03
 4.779e+03 4.779e+03 4.779e+03 4.779e+03 4.779e+03 4.160e+02 4.779e+03
 4.160

In [5]:
X_train, X_test, A_train, A_test = train_test_split(X, A, test_size=0.25, random_state=2137)

In [6]:
from tensorflow.keras.metrics import Metric, Precision, Recall

class F1Score(Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()
        self.f1_score = self.add_weight(name="f1", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)
        p = self.precision.result()
        r = self.recall.result()
        self.f1_score.assign(2 * ((p * r) / (p + r + 1e-6)))  # Added a small epsilon to avoid division by zero

    def result(self):
        return self.f1_score

    def reset_state(self):
        self.precision.reset_states()
        self.recall.reset_states()
        self.f1_score.assign(0)


In [7]:
model = Sequential([
    Dense(64, activation='relu', input_dim=num_features),
    Dense(64, activation='relu'),
    Dense(num_nodes, activation='sigmoid')  # Output layer
])

In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[
        BinaryAccuracy(name='accuracy'),
        Precision(name='precision'),
        Recall(name='recall'),
        AUC(name='auc'),
        F1Score()
    ])

In [25]:
model.fit(X_train, A_train, epochs=50, batch_size=10, validation_split=0.1)

Epoch 1/50
900/900 [==============================] - 7s 7ms/step - loss: 0.3756 - accuracy: 0.9972 - precision: 8.3467e-04 - recall: 0.0051 - auc: 0.5015 - f1_score: 0.0014 - val_loss: 0.0192 - val_accuracy: 0.9990 - val_precision: 0.0014 - val_recall: 0.0021 - val_auc: 0.4972 - val_f1_score: 0.0017
Epoch 2/50
900/900 [==============================] - 6s 6ms/step - loss: 0.0170 - accuracy: 0.9993 - precision: 0.0029 - recall: 0.0022 - auc: 0.5022 - f1_score: 0.0025 - val_loss: 0.0145 - val_accuracy: 0.9996 - val_precision: 0.0248 - val_recall: 7.7384e-04 - val_auc: 0.4993 - val_f1_score: 0.0015
Epoch 3/50
900/900 [==============================] - 6s 7ms/step - loss: 0.0105 - accuracy: 0.9995 - precision: 0.0132 - recall: 0.0025 - auc: 0.5045 - f1_score: 0.0041 - val_loss: 0.0113 - val_accuracy: 0.9995 - val_precision: 0.0053 - val_recall: 0.0012 - val_auc: 0.5054 - val_f1_score: 0.0019
Epoch 4/50
900/900 [==============================] - 6s 7ms/step - loss: 0.0089 - accuracy: 0.999

In [76]:
loss, accuracy = model.evaluate(X_test, A_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

84/84 [==============================] - 1s 12ms/step - loss: 0.0055 - accuracy: 0.9995 - precision: 0.3689 - recall: 0.1959 - auc: 0.6871 - f1_score: 0.2559


ValueError: too many values to unpack (expected 2)

In [52]:
def predict_adjacency_row(model, node_features):
    """
    Predict the adjacency row for a given node's features using a trained neural network model.
    
    Parameters:
    - model: The trained neural network model.
    - node_features: The feature vector for the node of interest (as a numpy array).
    
    Returns:
    - A binary vector representing the predicted connections from the node to all other nodes.
    """
    # Ensure node_features is the right shape: (1, num_features)
    node_features = np.array(node_features).reshape(1, -1)
    
    # Predict the connection probabilities using the model
    predicted_probabilities = model.predict(node_features)
    
    # Convert probabilities to binary values (0 or 1) using a threshold (e.g., 0.5)
    predicted_connections = (predicted_probabilities > 0.5).astype(int)
    
    # Return the first (and only) row of the predictions
    return predicted_connections[0]

# Example usage:
# Assuming you have the features for a specific node:
node_index = 10  # For example, if you want to predict connections for the 10th node
node_features = X[node_index]  # Extract features for this node

predicted_row = predict_adjacency_row(model, node_features)
print("Predicted connections for node:", predicted_row)
print("Real:", A[node_index, :])


1/1 [==============================] - 0s 11ms/step
Predicted connections for node: [0 0 1 ... 1 0 0]
Real: [0 1 1 ... 1 1 1]


In [6]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [22]:
def create_masks(num_nodes, train_percent=0.8, val_percent=0.1):
    """
    Randomly assign train, validation, and test masks for the nodes in the graph.
    
    Parameters:
    - num_nodes (int): Total number of nodes in the graph.
    - train_percent (float): Percentage of nodes to include in the training set.
    - val_percent (float): Percentage of nodes to include in the validation set.
    
    Returns:
    - Tuple of Tensors: (train_mask, val_mask, test_mask)
    """
    indices = np.random.permutation(num_nodes)
    train_size = int(num_nodes * train_percent)
    val_size = int(num_nodes * val_percent)

    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    train_mask[indices[:train_size]] = True
    val_mask[indices[train_size:train_size + val_size]] = True
    test_mask[indices[train_size + val_size:]] = True

    return train_mask, val_mask, test_mask

In [23]:
from torch_geometric.data import Data

def create_torch_geo_data(features, adjacency_matrix, num_nodes):
    edge_index = adjacency_to_edge_index(adjacency_matrix)
    x = features_to_tensor(features)
    train_mask, val_mask, test_mask = create_masks(num_nodes)
    data = Data(x=x, edge_index=edge_index, train_mask=train_mask, val_mask=val_mask, test_mask=test_mask)
    return data

In [24]:
import torch
import numpy as np

def adjacency_to_edge_index(adjacency_matrix):
    # Find the indices of nonzero elements (edges) in the adjacency matrix
    src, dst = adjacency_matrix.nonzero()
    # Create a tensor containing edge pairs
    edge_index = np.vstack([src, dst])
    return torch.tensor(edge_index, dtype=torch.long)

def features_to_tensor(features):
    return torch.tensor(features, dtype=torch.float)

haha = adjacency_to_edge_index(A)
print(haha)
hehe = features_to_tensor(X)
print(hehe)

tensor([[    0,     0,     0,  ..., 13325, 13325, 13325],
        [  674,   807,   873,  ...,  3501,  8144, 13226]])
tensor([[1.0000e+00, 3.0260e+03, 3.0260e+03,  ..., 8.7400e+02, 8.0800e+02,
         8.7400e+02],
        [4.7790e+03, 2.0000e+00, 4.1600e+02,  ..., 2.0000e+00, 3.4980e+03,
         3.4980e+03],
        [8.0810e+03, 7.1830e+03, 3.0000e+00,  ..., 7.1830e+03, 8.0810e+03,
         8.0810e+03],
        ...,
        [8.0400e+03, 4.6310e+03, 1.2823e+04,  ..., 8.0400e+03, 8.0400e+03,
         8.0400e+03],
        [1.9210e+03, 1.3325e+04, 1.3325e+04,  ..., 1.3325e+04, 1.3325e+04,
         1.3325e+04],
        [3.5020e+03, 3.5020e+03, 1.3227e+04,  ..., 3.5020e+03, 1.3326e+04,
         3.5020e+03]])


In [29]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class GCN(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Assuming num_classes is known, and features and adjacency_matrix are your numpy arrays
num_features = X.shape[1]
print(num_features)
num_classes = 13326  # You'll need to set this based on your dataset
data = create_torch_geo_data(X, A, num_classes)

# Initialize model
model = GCN(num_features, num_classes)

print(data)
# Training and evaluation code follows similar structure to previous example

128
Data(x=[13326, 128], edge_index=[2, 68562], train_mask=[13326], val_mask=[13326], test_mask=[13326])


In [30]:
import torch
from torch_geometric.datasets import Planetoid
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
print(data.x)  # Node features
print(data.edge_index)  # Edge list
print(data.y)  # Labels (NoneType error may be because this is missing)
print(data.train_mask)  # Training mask

tensor([[1.0000e+00, 3.0260e+03, 3.0260e+03,  ..., 8.7400e+02, 8.0800e+02,
         8.7400e+02],
        [4.7790e+03, 2.0000e+00, 4.1600e+02,  ..., 2.0000e+00, 3.4980e+03,
         3.4980e+03],
        [8.0810e+03, 7.1830e+03, 3.0000e+00,  ..., 7.1830e+03, 8.0810e+03,
         8.0810e+03],
        ...,
        [8.0400e+03, 4.6310e+03, 1.2823e+04,  ..., 8.0400e+03, 8.0400e+03,
         8.0400e+03],
        [1.9210e+03, 1.3325e+04, 1.3325e+04,  ..., 1.3325e+04, 1.3325e+04,
         1.3325e+04],
        [3.5020e+03, 3.5020e+03, 1.3227e+04,  ..., 3.5020e+03, 1.3326e+04,
         3.5020e+03]])
tensor([[    0,     0,     0,  ..., 13325, 13325, 13325],
        [  674,   807,   873,  ...,  3501,  8144, 13226]])
None
tensor([True, True, True,  ..., True, True, True])


In [28]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    # Optionally, evaluate on validation set
    model.eval()
    _, pred = out[data.val_mask].max(dim=1)
    correct = (pred == data.y[data.val_mask]).sum()
    val_accuracy = int(correct) / int(data.val_mask.sum())
    print(f'Epoch {epoch}, Val Accuracy: {val_accuracy:.4f}')

TypeError: 'NoneType' object is not subscriptable

In [9]:
model.eval()
_, pred = out[data.test_mask].max(dim=1)
correct = (pred == data.y[data.test_mask]).sum()
test_accuracy = int(correct) / int(data.test_mask.sum())
print(f'Test Accuracy: {test_accuracy:.4f}')

Accuracy: 0.7990


In [31]:
class GCN(torch.nn.Module):
    def __init__(self, num_features, hidden_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return x  # Output is node embeddings

In [32]:
def compute_edge_scores(node_embeddings):
    # Using dot product to score edges
    scores = torch.matmul(node_embeddings, node_embeddings.t())
    return torch.sigmoid(scores)  # Use sigmoid to map scores to probabilities

In [34]:
def create_edge_set(edge_index):
    edge_set = set()
    for i in range(edge_index.shape[1]):
        edge_set.add((edge_index[0, i].item(), edge_index[1, i].item()))
        edge_set.add((edge_index[1, i].item(), edge_index[0, i].item()))  # Add reverse direction for undirected graphs
    return edge_set

edge_set = create_edge_set(data.edge_index)

In [35]:
def get_negative_samples(edge_set, num_nodes, num_neg_samples):
    neg_edge_index = []
    while len(neg_edge_index) < num_neg_samples:
        i = np.random.randint(0, num_nodes)
        j = np.random.randint(0, num_nodes)
        if i != j and (i, j) not in edge_set:
            neg_edge_index.append([i, j])
    return torch.tensor(neg_edge_index).t()

negative_edge_index = get_negative_samples(edge_set, data.num_nodes, num_neg_samples=1000)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    embeddings = model(data)
    pos_scores = compute_edge_scores(embeddings)[data.edge_index[0], data.edge_index[1]]
    neg_scores = compute_edge_scores(embeddings)[negative_edge_index[0], negative_edge_index[1]]
    
    # Labels: 1s for positive samples, 0s for negative samples
    labels = torch.cat([torch.ones(pos_scores.shape[0]), torch.zeros(neg_scores.shape[0])])
    scores = torch.cat([pos_scores, neg_scores])
    
    loss = criterion(scores, labels)
    loss.backward()
    optimizer.step()